# Guided-MT Code2Vec Evaluation

This Notebook runs over the experiment-outputs, extracts data and makes plots.

Expected Layout:

```
.
├── README.md
├── data
│   └── random-MRR-max
│       ├── seed-2880
│       │   ├── data
│       │   │   ├── gen0
│       │   │   │   ├── 3b2459
│       │   │   │   ├── 3b2459.json
│       │   │   │   ├── 447e22
│       │   │   │   ├── 447e22.json
│       │   │   │   ├── 4495c7
│       │   │   │   ├── 4495c7.json
│       │   │   │   ├── 52667b
│       │   │   │   ├── 52667b.json
│       │   │   │   ├── 6855ba
│       │   │   │   ├── 6855ba.json
│       │   │   │   ├── 68ec75
│       │   │   │   ├── 68ec75.json
│       │   │   │   ├── 6cc14d
│       │   │   │   ├── 6cc14d.json
│       │   │   │   ├── 6d6845
│       │   │   │   ├── 6d6845.json
│       │   │   │   ├── 7a2d67
│       │   │   │   ├── 7a2d67.json
│       │   │   │   ├── ed0dd9
│       │   │   │   └── ed0dd9.json
│       │   │   ├── gen1
│       │   │   ├── ...
│       │   │   ├── gen8
│       │   │   ├── ...
│       │   │   ├── generation_0
│       │   │   │   ├── Some.java
│       │   │   │   ├── ...
│       │   │   │   ├── Other.java
│       │   │   │   └── Different.java
│       │   │   └── initialGen
│       │   │       └── 3bf9ce
│       │   └── results.txt
│       ├── seed-5142
│           └── results.txt
│       ...
├── evaluation.ipynb
└── requirements.txt
```

## Data Loading

Most of this is done in the nearby extract script, but we also extract some highlevel variables.

In [ ]:
import pandas as pd
import seaborn as sbn
import matplotlib.pyplot as plt
import extract

# Important: Specify Directory without / at the end!
directory:str = "./data"

In [ ]:
%%time
df = extract.make_df(directory)

In [ ]:
all_metrics = ["F1","MRR","EDITDIST","PMRR","REC","PREC"]
all_transformers = extract.get_known_transformers()
all_experiments = set(df["experiment"])
all_seeds = set(df["seed"])

In [ ]:
df.to_csv("./results.csv")
df.head(5)

## Per Experiment Plots

In [ ]:
broader_grouped_df = df.groupby(["experiment","generation"]).mean().reset_index()
broader_grouped_df["algorithm"] = broader_grouped_df["experiment"].apply(lambda x: "random" if "random" in x else "genetic")
broader_grouped_df.head(5)

In [ ]:
sbn.relplot(data=df,x="generation",y="F1", hue="algorithm")

In [ ]:
sbn.relplot(data=df,x="generation",y="MRR", hue="algorithm")

In [ ]:
sbn.relplot(data=broader_grouped_df,x="generation",y="F1", hue="experiment")

In [ ]:
sbn.relplot(data=broader_grouped_df,x="generation",y="F1", hue="experiment")

In [ ]:
df.info()

In [ ]:
fig, axs = plt.subplots(nrows=2,sharex=True)

sbn.lineplot(data=broader_grouped_df,x="generation",y="F1", style="algorithm",hue="experiment",ax=axs[0],legend=None,markers="o")
axs[0].set_title("MRR and F1 over experiments")
axs[0].set_ylim([0.35,0.7])
axs[0].set_xlim([0,26])

sbn.lineplot(data=broader_grouped_df,x="generation",y="MRR",style="algorithm", hue="experiment",ax=axs[1],markers="o")
axs[1].set_ylim([0.35,0.7])
axs[1].set_xlim([0,26])

#plt.legend(title="Experiment",bbox_to_anchor=(1.05, 2))
plt.legend(bbox_to_anchor=(1.05, 2))

plt.savefig("./figures/development-f1-mrr-all-experiments.png")
plt.show()

In [ ]:
helper_df = broader_grouped_df[broader_grouped_df["algorithm"]=="genetic"]
used_experiments = helper_df["experiment"].unique().to_list()
fig, axs = plt.subplots(nrows=2,sharex=True)

sbn.lineplot(data=helper_df,x="generation",y="F1",hue="experiment",ax=axs[0],legend=None,marker='o',hue_order=used_experiments)
axs[0].set_title("MRR and F1 over genetic experiments")
axs[0].set_ylim([0.35,0.7])

sbn.lineplot(data=helper_df,x="generation",y="MRR", hue="experiment",ax=axs[1],marker='o',hue_order=used_experiments)
axs[1].set_ylim([0.35,0.7])

plt.legend(title="Experiment",bbox_to_anchor=(1.45,1.45))
#plt.legend(bbox_to_anchor=(1.45, 1.45))

plt.savefig("./figures/development-f1-mrr-all-genetic-experiments.png")
plt.show()
del helper_df, used_experiments

In [ ]:
helper_df = broader_grouped_df[broader_grouped_df["algorithm"]=="genetic"]
used_experiments = helper_df["experiment"].unique().to_list()
sbn.lineplot(data=helper_df,x="generation",y="F1",hue="experiment",markers=True, hue_order=used_experiments)
plt.title("F1 over genetic experiments")
plt.ylim([0.35,0.55])
plt.xlim([0,20])

plt.savefig("./figures/development-f1-all-genetic-experiments.png")
plt.show()
del helper_df,used_experiments

In [ ]:
helper_df = broader_grouped_df[broader_grouped_df["algorithm"]=="genetic"]
used_experiments = helper_df["experiment"].unique().to_list()
sbn.lineplot(data=helper_df,x="generation",y="MRR",hue="experiment",markers=True, hue_order=used_experiments)
plt.title("MRR over genetic experiments")
plt.ylim([0.3,0.7])

plt.savefig("./figures/development-mrr-all-genetic-experiments.png")
plt.show()
del helper_df,used_experiments

In [ ]:
for exp in all_experiments:
    sbn.relplot(data=broader_grouped_df[broader_grouped_df["experiment"]==exp],x="generation",y="F1",kind="line")
    plt.title(f"F1 Score for {exp}")
    plt.show()


for exp in all_experiments:
    sbn.relplot(data=broader_grouped_df[broader_grouped_df["experiment"]==exp],x="generation",y="MRR",kind="line")
    plt.title(f"MRR Score for {exp}")
    plt.show()

## Notes

Things I saw / take away from plots:

- There are two interesting values, F1 and MRR both for F1-max.
- MRR seems to be shaky but not moving much, this could be because the model was trained with F1
- More movement with more generations
- There is a base-line jump for MRR min and max experiments?
- The F1 min maximizes MRR ...